# Test the three models

In [1]:
import keras
from keras import backend as K
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate, Add, Lambda
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D, MaxPooling2D
from keras.models import Sequential, Model
from keras.optimizers import Adam, RMSprop, Adadelta
from keras.applications import VGG19

import datetime
import matplotlib.pyplot as plt
import numpy as np
import scipy
import sys
import os

from data_helper import predict_15k, save_hist, save_model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, History

from time import gmtime, strftime
from data_helper import readImg, readImgInv, imagePatches, removeBlackImg, removeCorrespondence, check_and_create

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from mapgenlib.losses_extend import dice_coef_loss, iou_loss, wasserstein_loss
from mapgenlib.res_unit import res_block, decoder, encoder, build_res_unet
from mapgenlib.discrimnator import build_discriminator_simple_dc, build_discriminator_critic
from mapgenlib.discrimnator import build_discriminator_patchgan_srgan, build_discriminator_patchgan_cycle

from functools import partial
from keras.layers.merge import _Merge
class RandomWeightedAverage(_Merge):
    """Provides a (random) weighted average between real and generated image samples"""
    """Citation: https://github.com/eriklindernoren/Keras-GAN/blob/master/wgan_gp/wgan_gp.py#L110"""
    def _merge_function(self, inputs):
        alpha = K.random_uniform((32, 1, 1, 1))
        return (alpha * inputs[0]) + ((1 - alpha) * inputs[1])

In [3]:
class GAN4MapGen(): # Based on u-net, residual u-net and pix2pix
    # Reference: https://github.com/eriklindernoren/Keras-GAN/blob/master/pix2pix/pix2pix.py
    
    def __init__(self, method, batch_size=16):

        # Input shape
        self.img_rows = 128
        self.img_cols = 128
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        
        #self.clip_value = 0.01

        ## Calculate output shape of D (PatchGAN) better version
        #self.patch_size = 32
        #self.nb_patches = int((self.img_rows / self.patch_size) * (self.img_cols / self.patch_size))
        #self.patch_gan_dim = (self.patch_size, self.patch_size, self.channels)
        
        # Number of filters in the first layer of G and D
        self.gf = 64
        self.df = 64
        self.batch_size = batch_size
        
        self.method = method
        
        if method == "dc":
            self.initial_dc()
            self.train_cell = self.train_cell_dc
            self.valid_cell = self.valid_cell_dc
            
            # Adversarial ground truths
            self.valid = np.ones((batch_size, 1))
            self.fake = np.zeros((batch_size, 1))
        """
        if method == "wgangp":
            self.initial_wgangp()
            self.train_cell = self.train_cell_wgangp
            self.valid_cell = self.valid_cell_wgangp
            
            # Adversarial ground truths
            self.valid = -np.ones((batch_size, 1))
            self.fake =  np.ones((batch_size, 1))
        
        if method == "srgan":
            # Calculate output shape of D (PatchGAN)
            patch = int(self.img_rows / 2**4)
            self.disc_patch = (patch, patch, 1)
            
            self.initial_srgan()
            self.train_cell = self.train_cell_srgan
            self.valid_cell = self.valid_cell_srgan
            
            self.valid = np.ones((batch_size,) + self.disc_patch)
            self.fake = np.zeros((batch_size,) + self.disc_patch)
        
        if method == "p2p":
            # Calculate output shape of D (PatchGAN)
            patch = int(self.img_rows / 2**4)
            self.disc_patch = (patch, patch, 1)
            
            self.initial_p2p()
            self.train_cell = self.train_cell_p2p
            self.valid_cell = self.valid_cell_p2p
            
            self.valid = np.ones((batch_size,) + self.disc_patch)
            self.fake = np.zeros((batch_size,) + self.disc_patch)
        """
        
        """
        #optimizer = Adam(0.0002, 0.5) # Original
        #optimizer = Adam(0.0001, 0.5) # Original # Latest achieved by 0.00008
        #optimizer = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08) # An old version of Pix2pix
        optimizer = Adam(0.0004)
        
        #-------------------------
        # Construct Computational
        #   Graph of Generator
        #-------------------------

        # Build the generator
        #self.generator = self.build_generator() # Old generator from 
        self.generator = self.build_res_unet_generator()

        # Input images and their conditioning images
        #img_A = Input(shape=self.img_shape) # Target
        img_B = Input(shape=self.img_shape) # Input

        # By conditioning on B generate a fake version of A
        fake_A = self.generator(img_B)
        
        # Build and compile the discriminator
        self.discriminator = build_discriminator_simple_dc(self.img_shape)
        
        self.discriminator.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
        #self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        
        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # Discriminators determines validity of translated images / condition pairs
        #valid = self.discriminator([fake_A, img_B])
        #self.combined = Model(inputs=[img_A, img_B], outputs=[valid, fake_A])
        
        valid = self.discriminator(fake_A)
        self.combined = Model(inputs= img_B, outputs=[valid, fake_A])
        
        # Original Pix2Pix - low weight for discriminator
        self.combined.compile(loss=['mse', 'mae'], #['mse', 'mae'] original
                              loss_weights=[1, 100], # [1, 100] original
                              optimizer=optimizer, metrics=['accuracy'])
        """
    
    
    
    
    """ WGAN_GP 
    def gradient_penalty_loss(self, y_true, y_pred, averaged_samples):
    
        #Computes gradient penalty based on prediction and weighted real / fake samples
        gradients = K.gradients(y_pred, averaged_samples)[0]
        # compute the euclidean norm by squaring ...
        gradients_sqr = K.square(gradients)
        #   ... summing over the rows ...
        gradients_sqr_sum = K.sum(gradients_sqr,
                                  axis=np.arange(1, len(gradients_sqr.shape)))
        #   ... and sqrt
        gradient_l2_norm = K.sqrt(gradients_sqr_sum)
        # compute lambda * (1 - ||grad||)^2 still for each single sample
        gradient_penalty = K.square(1 - gradient_l2_norm)
        # return the mean as loss over all the batch samples
        return K.mean(gradient_penalty)
    
    def initial_wgangp(self):
        
        # Following parameter and optimizer set as recommended in paper
        self.n_critic = 5
        optimizer = RMSprop(lr=0.00005)

        # Build the generator and critic
        self.generator = build_res_unet(self.img_shape)
        self.critic = build_discriminator_critic(self.img_shape)
        
        #-------------------------------
        # Construct Computational Graph
        #       for the Critic
        #-------------------------------
        
        # Freeze generator's layers while training critic
        self.generator.trainable = False
        
        # Input images and generate imgs
        src_img = Input(shape=self.img_shape)
        real_img = Input(shape=self.img_shape)
        fake_img = self.generator(src_img)
        
        # Discriminator determines validity of the real and fake images
        fake = self.critic(fake_img)
        valid = self.critic(real_img)
        
        # Construct weighted average between real and fake images
        interpolated_img = RandomWeightedAverage()([real_img, fake_img])
        # Determine validity of weighted sample
        validity_interpolated = self.critic(interpolated_img)
        
        # Use Python partial to provide loss function with additional
        # 'averaged_samples' argument
        partial_gp_loss = partial(self.gradient_penalty_loss, averaged_samples=interpolated_img)
        partial_gp_loss.__name__ = 'gradient_penalty' # Keras requires function names

        self.critic_model = Model(inputs=[real_img, src_img], outputs=[valid, fake, validity_interpolated])
        self.critic_model.compile(loss=[wasserstein_loss, wasserstein_loss, partial_gp_loss],
                                        optimizer=optimizer,
                                        loss_weights=[1, 1, 10])
        #-------------------------------
        # Construct Computational Graph
        #         for Generator
        #-------------------------------

        # For the generator we freeze the critic's layers
        self.critic.trainable = False
        self.generator.trainable = True

        # Generate images based of noise
        fake_img = self.generator(src_img)
        # Discriminator determines validity
        valid = self.critic(fake_img)
        # Defines generator model
        self.generator_model = Model(src_img, [valid, fake_img])
        self.generator_model.compile(loss=[wasserstein_loss, "mse"], optimizer=optimizer)
        
    def train_cell_wgangp(self, imgs_A, imgs_B, valid, fake):
        
        dummy = np.zeros((self.batch_size, 1)) # Dummy gt for gradient penalty
        
        for _ in range(self.n_critic):

            # ---------------------
            #  Train Discriminator
            # ---------------------
            # Train the critic
            d_loss = self.critic_model.train_on_batch([imgs_A, imgs_B], [valid, fake, dummy])

        # ---------------------
        #  Train Generator
        # ---------------------
        g_loss = self.generator_model.train_on_batch(imgs_B, valid)
        return d_loss, g_loss
    
    def valid_cell_wgangp(self, x_test_sim, y_test_sim):
        num_test = len(x_test_sim)
        valid_test = np.ones((num_test,1))
        t_loss = self.generator_model.evaluate(x_test_sim, [valid_test, y_test_sim], verbose=0)
        return t_loss
    WGAN_GP ends""" 
    
    
    """ Pix2Pix 
    def initial_p2p(self):
        
        optimizer = Adam(0.0004)
        
        # Build the generator
        self.generator = build_res_unet(self.img_shape)
        
        # Build and compile the discriminator
        self.discriminator = build_discriminator_patchgan_cycle(self.img_shape, self.df)
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        
        # Input images and generate imgs
        img_B = Input(shape=self.img_shape)
        fake_A = self.generator(img_B)
        
        # For the combined model we will only train the generator
        self.discriminator.trainable = False
        
        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(fake_A)
        
        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(inputs=img_B, outputs=[valid, fake_A])
        self.combined.compile(loss=['binary_crossentropy', 'mse'], loss_weights=[1, 100], 
                              optimizer=optimizer, metrics=['accuracy'])
        #self.combined.compile(loss=['mse', 'mse'], loss_weights=[1, 100], 
        #                      optimizer=optimizer, metrics=['accuracy']) # Very bad try!!!
        
    def train_cell_p2p(self, imgs_A, imgs_B, valid, fake):
        # Condition on B and generate a translated version
        fake_A = self.generator.predict(imgs_B)
                
        # Train the discriminators (original images = real / generated = Fake)
        d_loss_real = self.discriminator.train_on_batch(imgs_A, valid)
        d_loss_fake = self.discriminator.train_on_batch(fake_A, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        # Train the generators
        g_loss = self.combined.train_on_batch(imgs_B, [valid, imgs_A])
        return d_loss, g_loss
    
    def valid_cell_p2p(self, x_test_sim, y_test_sim):
        num_test = len(x_test_sim)
        valid_test = np.ones((num_test,) + self.disc_patch)
        t_loss = self.combined.evaluate(x_test_sim, [valid_test, y_test_sim], verbose=0)
        return t_loss
    Pix2Pix ends""" 
    
    
    
    """ SRGAN 
    def initial_srgan(self):
        optimizer = Adam(0.0004)
        
        # We use a pre-trained VGG19 model to extract image features from the high resolution
        # and the generated high resolution images and minimize the mse between them
        self.vgg = self.build_vgg()
        self.vgg.trainable = False
        self.vgg.compile(loss='mse',optimizer=optimizer,metrics=['accuracy'])

        # Build and compile the discriminator
        self.discriminator = build_discriminator_patchgan_srgan(self.img_shape, self.df)
        self.discriminator.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
        
        # Build the generator
        self.generator = build_res_unet(self.img_shape)
        self.generator.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])

        # Input images and generate imgs
        img_A = Input(shape=self.img_shape)
        img_B = Input(shape=self.img_shape)
        
        # Generate image
        fake_A = self.generator(img_B)
        
        # Extract image features of the generated img
        fake_features = self.vgg(fake_A)
        
        # For the combined model we will only train the generator
        self.discriminator.trainable = False
        
        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(fake_A)
        
        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(inputs=[img_B, img_A], outputs=[valid, fake_features])
        self.combined.compile(loss=['binary_crossentropy', 'mse'], 
                              loss_weights=[1, 10], optimizer=optimizer, metrics=['mae'])
    
    def train_cell_srgan(self, imgs_A, imgs_B, valid, fake):
        # Condition on B and generate a translated version
        fake_A = self.generator.predict(imgs_B)
                
        # Train the discriminators (original images = real / generated = Fake)
        d_loss_real = self.discriminator.train_on_batch(imgs_A, valid)
        d_loss_fake = self.discriminator.train_on_batch(fake_A, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        valid = np.ones((self.batch_size,) + self.disc_patch)
        
        image_features = self.vgg.predict(imgs_A)
        
        # Train the generators
        g_loss = self.combined.train_on_batch([imgs_B, imgs_A], [valid, image_features])
        return d_loss, g_loss
    
    def valid_cell_srgan(self, x_test_sim, y_test_sim):
        num_test = len(x_test_sim)
        valid_test = np.ones((num_test,) + self.disc_patch)
        t_loss = self.generator.evaluate(x_test_sim, y_test_sim, verbose=0)
        return t_loss
     SRGAN ends"""
    
    
    
    
    """ DCGAN """
    def initial_dc(self):
        
        optimizer = Adam(0.0004)
        
        # Build the generator
        self.generator = build_res_unet(self.img_shape)
        
        # Build and compile the discriminator
        self.discriminator = build_discriminator_simple_dc(self.img_shape)
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        
        # Input images and generate imgs
        img_B = Input(shape=self.img_shape)
        fake_A = self.generator(img_B)
        
        # For the combined model we will only train the generator
        self.discriminator.trainable = False
        
        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(fake_A)
        
        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(inputs=img_B, outputs=[valid, fake_A])
        #self.combined.compile(loss=['binary_crossentropy', 'mse'], loss_weights=[1, 100], optimizer=optimizer, metrics=['accuracy'])
        
        #20190314 15:20
        self.combined.compile(loss=['mse', 'mae'], loss_weights=[1, 100], optimizer=optimizer, metrics=['accuracy'])
        
    def train_cell_dc(self, imgs_A, imgs_B, valid, fake):
        # Condition on B and generate a translated version
        fake_A = self.generator.predict(imgs_B)
                
        # Train the discriminators (original images = real / generated = Fake)
        d_loss_real = self.discriminator.train_on_batch(imgs_A, valid)
        d_loss_fake = self.discriminator.train_on_batch(fake_A, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        # Train the generators
        g_loss = self.combined.train_on_batch(imgs_B, [valid, imgs_A])
        return d_loss, g_loss
    
    def valid_cell_dc(self, x_test_sim, y_test_sim):
        num_test = len(x_test_sim)
        valid_test = np.ones((num_test,1))
        t_loss = self.combined.evaluate(x_test_sim, [valid_test, y_test_sim], verbose=0)
        return t_loss
    """ DC ends"""
    
    

    
    def train(self, x_train_sim, y_train_sim, x_test_sim, y_test_sim, outPath, epochs, sample_interval=50, patience = 5):
    
        start_time = datetime.datetime.now()
        
        valid = self.valid
        fake  = self.fake
        
        total_samples = len(x_train_sim)
        ids = np.arange(total_samples)
        np.random.shuffle(ids)
        n_batches = int(total_samples / self.batch_size)
        
        train_acc = []
        train_loss = []
        valid_acc = []
        valid_loss = []

        for epoch in range(epochs):
            for batch_i, (imgs_A, imgs_B) in enumerate(load_batch(x_train_sim, y_train_sim, self.batch_size)):
            
                d_loss, g_loss = self.train_cell(imgs_A, imgs_B, valid, fake)
            
                if batch_i % sample_interval == 0:
                    self.train_log(start_time, epoch, epochs, batch_i, n_batches, d_loss, g_loss)
                    print(g_loss)
            
            if epoch >= 0:
                t_loss = self.valid_cell(x_test_sim, y_test_sim)
                
                if self.method == "srgan":
                    self.valid_log_sr(start_time, epoch, epochs, batch_i, n_batches, d_loss, g_loss, t_loss)
                    
                    # Change to this if srgan
                    valid_loss.append(t_loss[0])
                    valid_acc.append(t_loss[1])
                else:
                    self.valid_log(start_time, epoch, epochs, batch_i, n_batches, d_loss, g_loss, t_loss)
                    
                    valid_loss.append(t_loss[2])
                    valid_acc.append(t_loss[4])
                
                train_loss.append(g_loss[2])
                train_acc.append(g_loss[4])
                
                waited = len(valid_loss) - 1 - np.argmin(valid_loss)
                print('waited for', waited, valid_loss)
                    
                if waited == 0:
                    self.generator.save(outPath + 'model_epoch'+ str(epoch) +'.h5')   
                        
                if waited > patience:
                    break
            
        return train_acc, train_loss, valid_acc, valid_loss
    
    def build_vgg(self):
        """
        Builds a pre-trained VGG19 model that outputs image features extracted at the
        third block of the model
        """
        vgg = VGG19(weights="imagenet")
        # Set outputs to outputs of last conv. layer in block 3
        # See architecture at: https://github.com/keras-team/keras/blob/master/keras/applications/vgg19.py
        vgg.outputs = [vgg.layers[9].output]

        img = Input(shape=self.img_shape)
        imgc = Concatenate()([img, img, img]) 

        # Extract image features
        img_features = vgg(imgc)

        return Model(img, img_features)

    
    def train_log(self, start_time, epoch, epochs, batch_i, n_batches, d_loss, g_loss):
        elapsed_time = datetime.datetime.now() - start_time
        print ("[Epoch %d/%d-%d/%d] [D loss&acc: %.3f, %.3f%%] [G loss&accA&accB: %.3f, %.3f%%, %.3f%%] time: %s" % (epoch, epochs,
                                                                                batch_i, n_batches,
                                                                                d_loss[0], 100*d_loss[1],
                                                                                100*g_loss[2], 100*g_loss[3], 100*g_loss[4],
                                                                                elapsed_time))    
    
    def valid_log_sr(self, start_time, epoch, epochs, batch_i, n_batches, d_loss, g_loss, t_loss):
        elapsed_time = datetime.datetime.now() - start_time
        print ("[Epoch %d/%d-%d/%d] [D loss&acc: %.3f, %.3f%%] [G loss&accA&accB: %.3f, %.3f%%, %.3f%%] time: %s" % (epoch, epochs,
                                                                                batch_i, n_batches,
                                                                                d_loss[0], 100*d_loss[1],
                                                                                100*g_loss[2], 100*g_loss[3], 100*g_loss[4],
                                                                                elapsed_time))   
        print(t_loss)
    
    def valid_log(self, start_time, epoch, epochs, batch_i, n_batches, d_loss, g_loss, t_loss):
        elapsed_time = datetime.datetime.now() - start_time
        print ("[Epoch %d/%d-%d/%d] [D loss&acc: %.3f, %.3f%%] [G loss&accA&accB: %.3f, %.3f%%, %.3f%%] [Test loss&acc: %.3f, %.3f%%, %.3f%%] time: %s" % (epoch, epochs,
                                                                                batch_i, n_batches,
                                                                                d_loss[0], 100*d_loss[1],
                                                                                100*g_loss[2], 100*g_loss[3], 100*g_loss[4],
                                                                                100*t_loss[2], 100*t_loss[3], 100*t_loss[4],
                                                                                elapsed_time))    

In [4]:
# Order the image dimension acc. to TensorFlow (batc_hsize, rows, cols, channels)
K.set_image_dim_ordering('tf')

scale = 15
p_size_1 = 128 # Compared with 256, which larger may generate round corners
trainPath = r"../tmp_data/data_feng_/geb" + str(scale) +  "/"

# save image patch arrays
x_train_sim = np.load(trainPath + "x_train_sim.npy")
y_train_sim = np.load(trainPath + "y_train_sim.npy")
x_test_sim = np.load(trainPath + "x_test_sim.npy")
y_test_sim = np.load(trainPath + "y_test_sim.npy")

input_shape1 = (None, None, 1) #x_train_sim[0].shape
print('Input Shape of the trains', x_train_sim.shape)
print('Input Shape of the tests', x_test_sim.shape)

Input Shape of the trains (32289, 128, 128, 1)
Input Shape of the tests (3587, 128, 128, 1)


In [5]:
def load_batch(x_train_sim, y_train_sim, batch_size):
    total_samples = len(x_train_sim)
    ids = np.arange(total_samples)
    np.random.shuffle(ids)
    n_batches = int(total_samples / batch_size)
    for i in range(n_batches-1):
        batch_idx = ids[i*batch_size:(i+1)*batch_size]
        imgs_A = x_train_sim[batch_idx]
        imgs_B = y_train_sim[batch_idx]
        yield imgs_B, imgs_A     
        
def load_data(x_test_sim, y_test_sim, batch_size=1):
    return x_test_sim  

def save_hist_local(train_acc, train_loss, valid_acc, valid_loss, outPath):    
    ### Save history
    History1_loss = train_loss
    History1_acc = train_acc
    History1_val_loss = valid_loss
    History1_val_acc = valid_acc

    thefile1 = open(outPath + 'History1_loss.txt', 'w')
    for item in History1_loss:
        thefile1.write("%s\n" % item)
    thefile1.close()

    thefile2 = open(outPath + 'History1_acc.txt', 'w')
    for item in History1_acc:
        thefile2.write("%s\n" % item)
    thefile2.close()

    thefile3 = open(outPath + 'History1_val_loss.txt', 'w')
    for item in History1_val_loss:
        thefile3.write("%s\n" % item)
    thefile3.close()

    thefile4 = open(outPath + 'History1_val_acc.txt', 'w')
    for item in History1_val_acc:
        thefile4.write("%s\n" % item)
    thefile4.close()

### Train residual unet + Simple

In [6]:
############ Path Setting ##############
outPath = r"../tmp_results/predictions/"
timestr = strftime("U"+str(p_size_1)+"GAN_%Y-%m-%d %H-%M-%S", gmtime())
outPath = outPath + timestr + '_' + str(scale)+ "/"
check_and_create(outPath)
print(outPath)

../tmp_results/predictions/U128GAN_2019-03-19 18-33-51_15/


In [7]:
gan = GAN4MapGen("dc", batch_size = 16)
train_acc, train_loss, valid_acc, valid_loss = gan.train(x_train_sim, y_train_sim, x_test_sim, y_test_sim, 
                                                         outPath, epochs=50, sample_interval=500)

/notebooks/tmp/MapGen/mapgenlib/res_unit.py:96: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  return Model(input=inputs, output=path)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_27 (Conv2D)           (None, 64, 64, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 64, 64, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 32, 32, 64)        18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 33, 33, 64)        0         
_________________________________________________________________
batch_normalization_25 (Batc (None, 33, 33, 64)        256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 33, 33, 64)        0         
__________

/usr/local/lib/python3.5/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


[Epoch 0/50-0/2018] [D loss&acc: 3.319, 6.250%] [G loss&accA&accB: 57.922, 100.000%, 31.525%] time: 0:00:11.883887
[57.93411, 0.012103392, 0.57922006, 1.0, 0.3152542]
[Epoch 0/50-500/2018] [D loss&acc: 1.021, 31.250%] [G loss&accA&accB: 1.518, 68.750%, 98.480%] time: 0:01:49.505618
[1.7664652, 0.24855304, 0.015179121, 0.6875, 0.98480225]
[Epoch 0/50-1000/2018] [D loss&acc: 0.730, 53.125%] [G loss&accA&accB: 1.205, 25.000%, 98.796%] time: 0:03:26.215028
[1.5894856, 0.38459238, 0.012048933, 0.25, 0.987957]
[Epoch 0/50-1500/2018] [D loss&acc: 0.873, 37.500%] [G loss&accA&accB: 1.459, 62.500%, 98.542%] time: 0:05:02.385781
[1.6791234, 0.22027695, 0.014588465, 0.625, 0.98542404]
[Epoch 0/50-2000/2018] [D loss&acc: 0.934, 43.750%] [G loss&accA&accB: 0.816, 43.750%, 99.184%] time: 0:06:38.197346
[1.1410089, 0.32497376, 0.008160352, 0.4375, 0.9918442]
[Epoch 0/50-2016/2018] [D loss&acc: 0.841, 37.500%] [G loss&accA&accB: 0.974, 50.000%, 99.026%] [Test loss&acc: 1.005, 64.065%, 98.996%] time: 0

[Epoch 7/50-1500/2018] [D loss&acc: 0.725, 50.000%] [G loss&accA&accB: 0.906, 37.500%, 99.095%] time: 0:50:42.114727
[1.1476023, 0.24202877, 0.009055736, 0.375, 0.9909477]
[Epoch 7/50-2000/2018] [D loss&acc: 0.764, 37.500%] [G loss&accA&accB: 0.615, 37.500%, 99.387%] time: 0:52:15.547389
[0.8928885, 0.2774368, 0.0061545167, 0.375, 0.99386597]
[Epoch 7/50-2016/2018] [D loss&acc: 0.753, 43.750%] [G loss&accA&accB: 0.885, 37.500%, 99.114%] [Test loss&acc: 0.690, 38.166%, 99.310%] time: 0:52:28.401117
waited for 0 [0.01004717296214241, 0.009768815507324568, 0.00934266658413637, 0.007916009215472392, 0.007559107356775653, 0.0072569459529268, 0.007082200821661495, 0.006903687165387999]
[Epoch 8/50-0/2018] [D loss&acc: 0.747, 34.375%] [G loss&accA&accB: 0.260, 75.000%, 99.739%] time: 0:52:28.960087
[0.4908646, 0.23103589, 0.002598287, 0.75, 0.99739456]
[Epoch 8/50-500/2018] [D loss&acc: 0.747, 53.125%] [G loss&accA&accB: 0.556, 37.500%, 99.443%] time: 0:54:02.292433
[0.8212394, 0.26538706, 0.

[Epoch 14/50-500/2018] [D loss&acc: 0.717, 40.625%] [G loss&accA&accB: 0.629, 56.250%, 99.372%] time: 1:32:38.003689
[0.8753767, 0.24676219, 0.0062861447, 0.5625, 0.9937172]
[Epoch 14/50-1000/2018] [D loss&acc: 0.764, 40.625%] [G loss&accA&accB: 0.855, 25.000%, 99.144%] time: 1:34:10.872384
[1.1306777, 0.2752741, 0.008554036, 0.25, 0.99144363]
[Epoch 14/50-1500/2018] [D loss&acc: 0.695, 50.000%] [G loss&accA&accB: 0.415, 25.000%, 99.585%] time: 1:35:43.715520
[0.7181388, 0.30336487, 0.004147739, 0.25, 0.9958496]
[Epoch 15/50-500/2018] [D loss&acc: 0.707, 53.125%] [G loss&accA&accB: 1.256, 25.000%, 98.744%] time: 1:39:02.282989
[1.5275645, 0.27107924, 0.012564852, 0.25, 0.987442]
[Epoch 15/50-1000/2018] [D loss&acc: 0.673, 62.500%] [G loss&accA&accB: 0.554, 50.000%, 99.445%] time: 1:40:35.223025
[0.8110098, 0.25691152, 0.005540983, 0.5, 0.99445343]
[Epoch 15/50-1500/2018] [D loss&acc: 0.673, 68.750%] [G loss&accA&accB: 0.231, 50.000%, 99.768%] time: 1:42:08.120294
[0.4634719, 0.2325035,

[Epoch 20/50-2016/2018] [D loss&acc: 0.712, 46.875%] [G loss&accA&accB: 0.648, 43.750%, 99.352%] [Test loss&acc: 0.658, 51.603%, 99.342%] time: 2:15:56.271554
waited for 0 [0.01004717296214241, 0.009768815507324568, 0.00934266658413637, 0.007916009215472392, 0.007559107356775653, 0.0072569459529268, 0.007082200821661495, 0.006903687165387999, 0.006868530663027882, 0.006796380373588803, 0.006862369040793194, 0.0067516930715747206, 0.006886414380612722, 0.0068582261454003465, 0.006767707003178223, 0.006698096887233706, 0.006697004151943129, 0.006605529214678982, 0.006667516233359526, 0.0067359845270565, 0.006575954473178376]
[Epoch 21/50-0/2018] [D loss&acc: 0.702, 50.000%] [G loss&accA&accB: 0.459, 31.250%, 99.541%] time: 2:15:56.866871
[0.7189609, 0.25991827, 0.004590426, 0.3125, 0.9954109]
[Epoch 21/50-500/2018] [D loss&acc: 0.728, 43.750%] [G loss&accA&accB: 0.488, 56.250%, 99.512%] time: 2:17:29.529744
[0.72087574, 0.23332322, 0.004875525, 0.5625, 0.9951248]
[Epoch 21/50-1000/2018] 

[Epoch 26/50-500/2018] [D loss&acc: 0.680, 62.500%] [G loss&accA&accB: 0.568, 68.750%, 99.432%] time: 2:49:29.055019
[0.8129293, 0.24506077, 0.0056786845, 0.6875, 0.9943199]
[Epoch 26/50-1000/2018] [D loss&acc: 0.684, 53.125%] [G loss&accA&accB: 0.461, 25.000%, 99.540%] time: 2:51:01.644231
[0.7246095, 0.26346684, 0.004611427, 0.25, 0.99539566]
[Epoch 26/50-1500/2018] [D loss&acc: 0.701, 40.625%] [G loss&accA&accB: 0.449, 62.500%, 99.551%] time: 2:52:34.289413
[0.69088554, 0.24217561, 0.0044870996, 0.625, 0.9955101]
[Epoch 26/50-2000/2018] [D loss&acc: 0.692, 46.875%] [G loss&accA&accB: 0.804, 43.750%, 99.195%] time: 2:54:07.092106
[1.0582613, 0.25395608, 0.008043052, 0.4375, 0.991951]
[Epoch 26/50-2016/2018] [D loss&acc: 0.703, 34.375%] [G loss&accA&accB: 0.195, 37.500%, 99.805%] [Test loss&acc: 0.664, 29.579%, 99.337%] time: 2:54:19.835604
waited for 6 [0.01004717296214241, 0.009768815507324568, 0.00934266658413637, 0.007916009215472392, 0.007559107356775653, 0.0072569459529268, 0.00

In [8]:
save_hist_local(train_acc, train_loss, valid_acc, valid_loss, outPath)